# NLP with Logistic Regression

Les 4 différentes catégories sont :
- peur pour eux
- peur pour les autres
- peur sur la gestion de la crise
- fourre tout

## Extraction et preprocessing des données

In [1]:
import pandas as pd
import numpy as np
from treat_data import treat_data

DIRECTORY = "challenge_dataset/"

# extract data
X = pd.read_csv(DIRECTORY + 'X_train.csv', sep=';').drop(columns=['Id'])
y = pd.read_csv(DIRECTORY + 'y_train.csv', sep=';').drop(columns=['Id'])

# preprocessing data
X_clean = pd.DataFrame()
X_clean['lemmas'], X_clean['pos'] = treat_data(X)

X_clean

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\delan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\delan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\Users\delan\anaconda3\lib\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'fr_core_news_md' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.3.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


,lemmas,pos
0,"[mourir, heure, plus, revoir, petit, enfant]","[VERB, NOUN, ADV, VERB, ADJ, NOUN]"
1,"[maladie, conséquence, jeune, génération, voir...","[NOUN, NOUN, ADJ, NOUN, VERB, NOUN, ADJ, NOUN,..."
2,"[sortir, mal, loger]","[VERB, ADV, VERB]"
3,"[inquiétude, santé, proche, plus, fragile, fem...","[NOUN, ADJ, ADJ, ADV, ADJ, NOUN, VERB, NOUN, N..."
4,"[bien, entendre, contracter, maladie]","[ADV, VERB, VERB, NOUN]"
...,...,...
480,"[forme, grave, civid]","[NOUN, ADJ, ADJ]"
481,"[inquiétude, retrouver, liberté, action, total...","[NOUN, VERB, NOUN, NOUN, ADJ, VERB, NOUN, NOUN..."
482,"[incertitude, lequel, plus, voir, petit, fils,...","[NOUN, PRON, ADV, VERB, ADJ, NOUN, ADP, NOUN, ..."
483,"[inquiétude, normal, face, épidémie, crainte, ...","[NOUN, ADJ, NOUN, NOUN, NOUN, VERB, ADJ, NOUN,..."


In [2]:
y.sum()

category_1    135
category_2    172
category_3    237
category_4     57
dtype: int64

category_4 est une catégorie "fourre-tout". De plus, on possède moins de données sur cette catégorie que les autres. Il pourrais donc essayer d'entraîner le jeu de données sans prendre en compte cette catégorie, puis définir cette catégorie comme label par défaut

## Mise en forme des données pour la regression logistique

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = X_clean['lemmas'].apply(lambda x: ' '.join(x)).to_list()
labels = y.columns

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

## Mise en place du modèle

In [4]:
from sklearn.linear_model import LogisticRegression

LOGISTIC_REGRESSION_PARAMETERS = {'C': 0.1,
                                  'max_iter': 100}

def fit_one_classifier(X, y):
    classifier = LogisticRegression(**LOGISTIC_REGRESSION_PARAMETERS)
    return classifier.fit(X, y)

def fit_all_classifiers(X, y_full, labels):
  classifiers = {}
  for idx, label in enumerate(labels):
      target = y_full[label]
      classifier = fit_one_classifier(X, target)
      classifiers[label] = classifier
  return classifiers

clfs = fit_all_classifiers(X, y, labels)

## Score f1 du modèle

In [5]:
from sklearn.model_selection import cross_val_score


def cross_val_score_classifier(X, y):
  classifier = LogisticRegression(**LOGISTIC_REGRESSION_PARAMETERS)
  cv_score = np.mean(cross_val_score(classifier, X, y, scoring='f1'))
  return cv_score

def compute_CV_score_for_each_class(X, y_full, labels):
  scores = []
  for label in labels:
      target = y_full[label].values
      cv_score = cross_val_score_classifier(X, target)
      scores.append(cv_score)
  return scores

scores = compute_CV_score_for_each_class(X, y, labels)
for idx in range(len(scores)):
  print("f1 score pour la", labels[idx], ":", scores[idx])

f1 score pour la category_1 : 0.1601514910747391
f1 score pour la category_2 : 0.6212450536216092
f1 score pour la category_3 : 0.7704773846842813
f1 score pour la category_4 : 0.0


Les résultats sont assez bons pour les catégories 2 et 3 mais très mauvais pour les catégories 1 et 4. Pour la catégorie 4 celà est peut-être dû au manque de données.

## Prédictions pour Kaggle

In [7]:
# extract data
X = pd.read_csv(DIRECTORY + 'X_test.csv', sep=';')
index = X['Id'] # save index for submission

# preprocessing data
X_clean = pd.DataFrame()
X_clean['lemmas'], X_clean['pos'] = treat_data(X)
X_clean.insert(0, 'Id', index)

# vectorization
corpus = X_clean['lemmas'].apply(lambda x: ' '.join(x)).to_list()
X = vectorizer.transform(corpus)

# prediction
y_pred = pd.DataFrame.from_dict({label: clf.predict(X) for label, clf in clfs.items()})
y_pred.insert(0, 'Id', index)

In [ ]:
X_clean.to_csv("challenge_dataset/X_test_clean.csv", sep=';', index=False)

In [100]:
y_pred.to_csv("results/y_logistic_regression_on_labels_066968.csv", index=False)